In [35]:
import pandas as pd
import numpy as np
import pickle
from pandas import ExcelWriter
import ffn
%matplotlib inline

def save_xls(list_dfs, xls_path,sheet_names):
    writer = ExcelWriter(xls_path)
    for n, df in enumerate(list_dfs):
        df.to_excel(writer, sheet_names[n])
    writer.save()
    return

#pull in data from AlphaVantage
from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries(key='NXY0VT9AHBRYGKKC',output_format='pandas')

def historical_data(ticker, outsize = "full"):
    #outsize can be compact and full. 
    #compact returns only the latest 100 data points; full 
    #returns the full-length time series of up to 20 years of historical data. The "compact" option is 
    #recommended if you would like to reduce the data size of each API call
    alphavantage_link = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={0}&apikey=NXY0VT9AHBRYGKKC&datatype=csv&outputsize={1}'.format(ticker, outsize)
    df = pd.read_csv(alphavantage_link)
    return df

In [36]:
EQ = ['DIS', 'NFLX']
Maindf = historical_data("SPY")
Maindf.index = pd.to_datetime(Maindf.timestamp) #set the index as the timestamp
#Maindf.index = pd.to_datetime(Maindf.index) #make the index datetime
Maindf = Maindf[['timestamp','close']]
Maindf.columns = ["timestamp","SPY"]

for ticker in EQ:
    temp_df = historical_data(ticker)
    temp_df.index = pd.to_datetime(temp_df.timestamp) #set the index as the timestamp
    temp_df = temp_df[['close']]
    temp_df.columns = [ticker]
    Maindf = Maindf.join(temp_df, how = "left")

Maindf.dropna(inplace = True)
Maindf = Maindf.sort_index()



In [37]:
del Maindf['timestamp']


In [38]:
perf = Maindf.calc_stats()

C:\Users\scheng\AppData\Local\Continuum\anaconda3\lib\site-packages\ffn\core.py:2054: RuntimeWarning: invalid value encountered in minimum
  negative_returns = np.minimum(returns, 0.)


In [39]:
perf.display()

Stat                 SPY         DIS         NFLX
-------------------  ----------  ----------  ----------
Start                2002-05-23  2002-05-23  2002-05-23
End                  2018-07-10  2018-07-10  2018-07-10
Risk-free rate       0.00%       0.00%       0.00%

Total Return         153.22%     344.84%     2388.42%
Daily Sharpe         0.40        0.48        0.71
Daily Sortino        0.63        0.80        1.08
CAGR                 5.93%       9.69%       22.05%
Max Drawdown         -56.47%     -57.35%     -88.37%
Calmar Ratio         0.10        0.17        0.25

MTD                  2.77%       1.14%       6.48%
3m                   5.14%       4.57%       39.84%
6m                   1.70%       -3.17%      96.13%
YTD                  4.47%       -1.40%      117.13%
1Y                   15.03%      2.33%       173.01%
3Y (ann.)            10.35%      -3.08%      -15.08%
5Y (ann.)            11.04%      10.31%      11.32%
10Y (ann.)           8.33%       13.61%      31.08%
Si

In [40]:
perf.stats

,DIS,NFLX,SPY
start,2002-05-23 00:00:00,2002-05-23 00:00:00,2002-05-23 00:00:00
end,2018-07-10 00:00:00,2018-07-10 00:00:00,2018-07-10 00:00:00
rf,0,0,0
total_return,3.44838,23.8842,1.53215
cagr,0.0969393,0.220491,0.0592846
max_drawdown,-0.573461,-0.883724,-0.564737
calmar,0.169043,0.249502,0.104977
mtd,0.0114016,0.0648415,0.0276836
three_month,0.0457236,0.398366,0.0514426
six_month,-0.0316525,0.961278,0.0170363
